<a href="https://colab.research.google.com/github/srujaan/DL-with-PyTorch/blob/master/UsingTorchNNmodule.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
t_c = [0.5, 14.0, 15.0, 28.0, 11.0, 8.0, 3.0, -4.0, 6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]

# We reshape our B inputs to B × Nin, 
# where Nin is 1. That is easily done with unsqueeze :
# to use nn.lineaer

t_c = torch.tensor(t_c).unsqueeze(1) # Adds the extra dimension at axis 1
t_u = torch.tensor(t_u).unsqueeze(1) # Adds the extra dimension at axis 1

t_u.shape, t_c.shape

(torch.Size([11, 1]), torch.Size([11, 1]))

In [8]:
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_indices, val_indices

(tensor([ 4,  1,  0,  6,  8,  3,  9,  7, 10]), tensor([2, 5]))

In [9]:
t_u_train = t_u[train_indices]
t_c_train = t_c[train_indices]

t_u_val = t_u[val_indices]
t_c_val = t_c[val_indices]

t_un_train = 0.1 * t_u_train
t_un_val = 0.1 * t_u_val


In [10]:
# update the training code with nn.linear

linear_model = nn.Linear(1, 1)
optimizer = optim.SGD(
    linear_model.parameters(), # This method call replaces[params]
    lr = 1e-2)

In [11]:
# Now we can use the parameters method to ask any nn.Module for
# a list of parameters owned by it or any of its submodules:

linear_model.parameters()

<generator object Module.parameters at 0x7fc11dbe3f10>

In [12]:
list(linear_model.parameters())

[Parameter containing:
 tensor([[0.0297]], requires_grad=True), Parameter containing:
 tensor([0.0630], requires_grad=True)]

In [13]:
# Training loop

def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val, 
                  t_c_train, t_c_val):
  for epoch in range(1, n_epochs + 1):
    t_p_train = model(t_u_train) # The model is now passed in, instead of params
    loss_train = loss_fn(t_p_train, t_c_train)

    t_p_val = model(t_u_val) # The model is now passed in, instead of params

    loss_val = loss_fn(t_p_val, t_c_val)

    optimizer.zero_grad()
    loss_train.backward() # The loss function is also passed in. We'll use it in a moment
    optimizer.step()

    if epoch == 1 or epoch % 1000 == 0:
      print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
      f" Validation loss {loss_val.item():.4f}")


In [14]:
linear_model = nn.Linear(1, 1) # <1>
linear_model(t_un_val)

tensor([[-1.3435],
        [-1.2310]], grad_fn=<AddmmBackward>)

In [16]:
linear_model = nn.Linear(1, 1)
optimizer = optim.SGD(linear_model.parameters(), lr=1e-2)

training_loop(
    n_epochs = 3000,
    optimizer = optimizer,
    model = linear_model,
    loss_fn = nn.MSELoss(), # we no longer using our hand written loss function from earlier
    t_u_train = t_un_train,
    t_u_val = t_un_val,
    t_c_train = t_c_train,
    t_c_val = t_c_val)

print()
print(linear_model.weight)
print(linear_model.bias)

Epoch 1, Training loss 87.7353, Validation loss 45.5313
Epoch 1000, Training loss 3.9171, Validation loss 1.4668
Epoch 2000, Training loss 3.3592, Validation loss 1.0954
Epoch 3000, Training loss 3.3497, Validation loss 1.0533

Parameter containing:
tensor([[5.3263]], requires_grad=True)
Parameter containing:
tensor([-17.1012], requires_grad=True)
